### Imposing Model Constraints 

Here we take measured gas exchange, metabolite and proteomics data in order to constraint the flux of the reaction in the model by [Arnold and Nikoloski, 2014](https://www.ncbi.nlm.nih.gov/pubmed/24808102).

We build four models using the experimental data: 
- Wild-type 20 degrees,
- Wild-type after 7 days in 4 degrees, 
- fum2 mutant 20 degrees, 
- fum2 mutant after 7 days in 4 degrees. 

In [1]:
# # Loading Required Packages 
# import cobra
# from cobra.flux_analysis import flux_variability_analysis
# from cobra.flux_analysis.loopless import add_loopless, loopless_solution
# from cobra.flux_analysis import pfba
# import pandas as pd
# import numpy as np
# import itertools

In [2]:
# Loading the protein data 
df = pd.read_excel(io="ProteinConc.xlsx")

In [3]:
# Loading the model 
%run SetUpModel.ipynb

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f7033080c90>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f7032b44450>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f7033080e10>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 
G6P_c + Pi_h --> G6P_h + Pi_c
G6P_c + Pi_h <=> G6P_h + Pi_c
(-1000.0, 1000.0)
Set to be bidirectional. Dyson et al. 2015 

PGA_h + Pi_c <=> PGA_c + Pi_h
(0.01, 1000.0)
Set as one directional to avoid futile cycles
GAP_h + Pi_c <=> GAP_c + Pi_h
(0.01, 1000.0)
Set as one directional to avoid futile cycles


In [4]:
# Functions for running the models 

def ATPNADPH(atp,nadph):
    '''Setting the ATP and NADPH flux. Lower bound plus one'''
    NADPH = model.reactions.get_by_id('Fd_DASH_NADPR_h')
    ATPase = model.reactions.get_by_id('ATPase_h')
    lohi = 1.0
    NADPH.lower_bound = nadph - lohi
    NADPH.upper_bound = nadph + lohi
    ATPase.lower_bound = atp- lohi
    ATPase.upper_bound = atp + lohi
    print(NADPH)
    print(NADPH.bounds)
    print(ATPase)
    print(ATPase.bounds)
    print(" ")
    
def setobjective(FunName,obj="Maximum"):
    '''Setting the objective function of the model to FunName'''
    model.objective = model.reactions.get_by_id(FunName)
    model.objective_direction = obj
    print(model.objective)
    print("Set as objective function. \n")
    
def constr(reacs,Dat1,Dat2,Dat3,Dat4):
    '''Constraints model reactions for a specific genotype and temperature'''
    for r in reacs:
        upper_vals = []
        lower_vals = []
        for g in r.genes:
            # Get index of protein in data file 
            ind = all_atg.index(g.id)
            # Scaling the values for setting fluxes 
            vals = [i/4000. for i in Dat1[ind],Dat2[ind],Dat3[ind],Dat4[ind]]
            val = np.mean(vals)
            sterr = np.std(vals)/np.sqrt(len(vals))
            upper = val + sterr
            lower = val - sterr
            upper_vals.append(upper)
            lower_vals.append(lower)
        # Constraint flux of forward reaction
        r.upper_bound = round(sum(upper_vals),3)
        # Constraint flux of lower reaction 
        if r.lower_bound < 0.0:
            r.lower_bound = round(-sum(upper_vals),3)
        # No lower bound b/c don't know localization or enzymes are used to in other reactions 

def TPTratio(TPT1,TPT2):
    '''To set TPT1 to TPT2 flux ratios'''
    r = model.reactions.get_by_id('Tr_TPT2')
    r.lower_bound = 0.0
    r.upper_bound = TPT2
    r = model.reactions.get_by_id('Tr_TPT1')
    r.lower_bound = 0.0
    r.upper_bound = TPT1
    #print(cobra.flux_analysis.flux_variability_analysis(model, [NADPH,ATPase]))
    print(" ")
    #inds = (1,101,202,302,401,501,601,701,801,901)
    
def subsys(subsystem):
    for r in model.reactions:
        if r.subsystem == subsystem:
            print(r.reaction)


In [5]:
# # Setting the ATP and NADPH production fluxes 
# setobjective("FumHA_c")
# ATPNADPH(107.0,160.0) #0.668
# #ATPNADPH(97.0,220.0) #0.441
# flux_variability_analysis(model,["Tr_TPT1","Tr_TPT2"])
# #subsys("light reactions")

In [6]:
# Get list of all possible proteins in the model
gene_ids = []
for g in model.genes:
    gene_ids.append(g.id)
# Get list of all measured proteins 
all_atg = []
x=df["Accession"]
for atg in x.values:
    all_atg.append(atg)
# Comparing the measured protein concentrations to those which are in the model 
model_atg = [] # Total measured proteins in model
model_atg_final = [] # Proteins with direct reaction constraint (i.e. single responsible protein)
model_rxns_atg = [] # Model reactions with direct constraints (i.e. single responsible protein)
model_rxns_atgs = [] # Model reactions with multiple possible constraints 
model_atgs_final = [] # Proteins with multiple possible constraints 

for atg in x.values: # Check if protein is in the model 
    if atg in gene_ids:
        model_atg.append(atg)
        # Only parameterize that reaction if that enzyme is the only one responsible for that reaction
        for r in model.genes.get_by_id(atg).reactions:
            if len(r.genes) == 1:
                model_rxns_atg.append(r)
                model_atg_final.append(atg)
            else:
                prot_names = []
                for g in r.genes:
                    prot_names.append(g.id)
                if set(prot_names).issubset(set(all_atg)):
                    model_rxns_atgs.append(r)
                    model_atgs_final = model_atgs_final + prot_names
# Feasible constraint summary 
print("Total measured proteins: {}".format(len(all_atg)))  
print("Total measured proteins in model: {}".format(len(model_atg))) 
print("Total model reactions: {}".format(len(model.reactions)))
print(" ")
# Reactions with simple constraints 
print("Total model reactions with single constraints: {}".format(len(set(model_rxns_atg))))
print("Model proteins for single constraints: {}".format(len(set(model_atg_final))))
print(" ")
# Reactions with multiple possible constraints 
print("Total model reactions with multiple constraints: {}".format(len(set(model_rxns_atgs))))
print("Total model proteins for possible constraints: {}".format(len(set(model_atgs_final))))
# Total constained reactions
constr_reacs = set(model_rxns_atg + model_rxns_atgs) #these are the reactions constrained in the models 
constr_ids = []
for r in constr_reacs:
    constr_ids.append(r.id)

Total measured proteins: 2427
Total measured proteins in model: 245
Total model reactions: 552
 
Total model reactions with single constraints: 62
Model proteins for single constraints: 49
 
Total model reactions with multiple constraints: 39
Total model proteins for possible constraints: 53


In [7]:
# Setting up the Model Constraints
# for different genotypes and temperatures 

def all_constr(C,M,F,S,Cond):
    # Metabolite Constraints 
    print("\n--------- {} -----------\n".format(Cond))
    f1=0.99*200
    f2=1.01*200
    rMal = model.reactions.get_by_id('Mal_Store')
    rFum = model.reactions.get_by_id('Fum_Store')
    rStarch = model.reactions.get_by_id('Starch_Store')
    rCO2 = model.reactions.get_by_id('Im_CO2')
    FumHA_c = model.reactions.get_by_id('FumHA_c')
    rResp = model.reactions.get_by_id('Tr_Pyr2')
    rCO2.upper_bound = C+1.0
    rCO2.lower_bound = C-1.0
    rResp.lower_bound = rCO2.lower_bound*0.15/3
    rResp.upper_bound = 1000.0
    rMal.upper_bound = (M+0.001)*f2
    rMal.lower_bound = (M-0.001)*f1
    rFum.upper_bound = (F+0.001)*f2
    rFum.lower_bound = (F-0.001)*f1
    rStarch.upper_bound = (S-0.001)*f2
    rStarch.lower_bound = (S-0.001)*f1
    # Proteomic Constraints 
    name1 = "{} 1".format(Cond)
    name2 = "{} 2".format(Cond)
    name3 = "{} 3".format(Cond)
    name4 = "{} 4".format(Cond)
    constr(constr_reacs,df[name1],df[name2],df[name3],df[name4])
    # Mutant Knockout
    if "fum2" in Cond:
        r = model.reactions.get_by_id('FumHA_c')
        r.upper_bound = 0.0 # Mutant knockout 
        r.lower_bound = 0.0 
    # Loopless Constraints 
    print("For Loopless...")
    setobjective("Mal_Store",obj="Minimum")
    loopless = loopless_solution(model)
    print(loopless["Tr_TPT1"])
    print(loopless["Tr_TPT2"])
    print(" ")
    for r in model.reactions:
        if (loopless[r.id] == 0.0):
            r.lower_bound = 0.0
            r.upper_bound = 0.0
        else:
            pass 
    FVA = flux_variability_analysis(model,['Fd_DASH_NADPR_h'])
    modelname = "Arnold2014_{}_Loopless.xml".format(Cond)
    cobra.io.write_sbml_model(model,modelname,use_fbc_package=False)
    print("MODEL SAVED. Go do flux sampling in MATLAB. \n")
    # Resetting the mutant knockout 
    if "fum2" in Cond:
        r = model.reactions.get_by_id('FumHA_c')
        r.upper_bound = 1000.0 
        r.lower_bound = 0.0


In [8]:
# Parameterizing the model for Genotype and Condition
# all_constr(100,0.003,0.006,0.028,"WT Cntl")
# all_constr(100,0.012,0.0,0.042,"fum2 Cntl")
# all_constr(100,0.012,0.023,0.051,"WT cold")
# all_constr(94,0.025,0.0,0.063,"fum2 Cold")

--------- fum2 Cold -----------

For Loopless...
Minimize
-1.0*Mal_Store_reverse_8006f + 1.0*Mal_Store
Set as objective function. 

0.01
39.45111111111003
 
MODEL SAVED. Go do flux sampling in MATLAB. 



In [9]:
# CHRR flux sampling (MATLAB SCRIPT)
# Plotting the overlapping sampling distribution (NEW PYTHON SCRIPT)